## Demonstrate how to set up file tree so the ig-pub can create the page heirarchy correctly in the IG resource and flatted the tree for Jekyll

so from this ...

~~~
.
├── Downloads
│   └── index.md
├── Examples
│   └── index.md
├── FHIR\ Artifacts
│   ├── Capability\ Statements
│   │   └── index.md
│   ├── Operations\ and\ Search\ Parameters
│   │   └── index.md
│   ├── Profiles\ and\ Extensions
│   │   └── index.md
│   └── Terminology
│       └── index.md
├── Guidance
│   ├── General\ Guidance
│   │   └── index.md
│   └── More\ Guidance
│       └── index.md
├── Security
│   └── index.md
├── images
├── includes
└── index.md
~~~

to this ...

~~~
.
├── Capability\ Statements.md
├── Downloads.md
├── Examples.md
├── General\ Guidance.md
├── More\ Guidance.md
├── Operations\ and\ Search\ Parameters.md
├── Profiles\ and\ Extensions.md
├── Security.md
├── Terminology.md
└── index.md

~~~

In [21]:
import os
import sys
#import string
import shutil
from pprint import pprint

### Generate the file paths to traverse, or a single path if a file name was given

In [28]:
def getfiles(path):
    if os.path.isdir(path):
        for root, dirs, files in os.walk(path):
            print('*'*25)
            print(f'root= {root}')
            print(f'dirs= {dirs}')
            print(f'files= {files}')
            print('*'*25)
            yield os.path.join(root, name)
    else:
        yield

### Generate the menu dictionary for making html menu

In [208]:
def path_to_dict(path, d):

    name = os.path.basename(path)  

    if os.path.isdir(path):
        if name not in d['dirs']:
            d['dirs'][name] = {'dirs':{}}
        for x in os.listdir(path):
            path_to_dict(os.path.join(path,x), d['dirs'][name])
    return d

### get source folder

In [179]:
fromdir = "/Users/ehaas/Documents/FHIR/test_template/pages"
#fromdir = input('enter the source folder: ')
print(fromdir)

/Users/ehaas/Documents/FHIR/test_template/pages


### get destination folder

In [180]:
destination = "/Users/ehaas/Documents/FHIR/test_template/test"
#destination = input('enter the destination folder: ')
print(destination)

/Users/ehaas/Documents/FHIR/test_template/test


### First Clear the directory

In [181]:
for filename in os.listdir(destination):
    file_path = os.path.join(destination,filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

### Create Dict to make html menu

In [211]:
tree = path_to_dict(fromdir, d = {'dirs':{}})


pprint(tree)

{'dirs': {'pages': {'dirs': {'Downloads': {'dirs': {}},
                             'Examples': {'dirs': {}},
                             'FHIR Artifacts': {'dirs': {'Capability Statements': {'dirs': {}},
                                                         'Operations and Search Parameters': {'dirs': {}},
                                                         'Profiles and Extensions': {'dirs': {}},
                                                         'Terminology': {'dirs': {}}}},
                             'Guidance': {'dirs': {'General Guidance': {'dirs': {}},
                                                   'More Guidance': {'dirs': {}}}},
                             'Security': {'dirs': {}},
                             'images': {'dirs': {}},
                             'includes': {'dirs': {}}}}}}


### Use Jinja2 template to create html

In [242]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown

template_path = 'menu_html.j2'
in_path =''

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(template_path)

d = template.render(tree=tree)
display(HTML(d))

print(d)

<!-- Create Menu xhtml from directory -->
<!doctype xhtml>
<title>Menu from File Tree</title>
<h1>Menu</h1>
<ul>
      <li><a href="index.md">Home</a></li>
      <li><a href="Security.md">Security</a></li>
      <li><a href="images.md">images</a></li>
      <li><a href="Guidance.md">Guidance</a></li>
       <ul>
       
      <li><a href="More Guidance.md">More Guidance</a></li>
      <li><a href="General Guidance.md">General Guidance</a></li>
       </ul>
      <li><a href="includes.md">includes</a></li>
      <li><a href="FHIR Artifacts.md">FHIR Artifacts</a></li>
       <ul>
       
      <li><a href="Operations and Search Parameters.md">Operations and Search Parameters</a></li>
      <li><a href="Terminology.md">Terminology</a></li>
      <li><a href="Capability Statements.md">Capability Statements</a></li>
      <li><a href="Profiles and Extensions.md">Profiles and Extensions</a></li>
       </ul>
      <li><a href="Examples.md">Examples</a></li>
      <li><a href="Downloads.md">D

### Flatten Directory and Replace 'index.md' with [Directory name].md

In [238]:
for f in getfiles(fromdir):
    print(f'f={f}')
    filename = f.split('/')[-1]
    
    if filename != '.DS_Store':
        new_filepath = os.path.join(destination,filename)
        print(f'new_filepath before check: {new_filepath}')
        if os.path.isfile(new_filepath):
            filename = f.split('/')[-2] + '.md'
        new_filepath = os.path.join(destination,filename)
        print(f'new_filepath after check: {new_filepath}')
        shutil.copy(f, new_filepath)

*************************
root= /Users/ehaas/Documents/FHIR/test_template/pages
dirs= ['Security', 'images', 'Guidance', 'includes', 'FHIR Artifacts', 'Examples', 'Downloads']
files= ['.DS_Store', 'index.md']
*************************


NameError: name 'name' is not defined